**PyTorch**

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)

test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(7 * 7 * 64, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = FashionCNN()
print(model)

entr_loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
EPOCHS = 10

for epoch in range(1, EPOCHS + 1):
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = entr_loss(output, target)
        loss.backward()
        optimizer.step()

    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += entr_loss(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    avg_test_loss = test_loss / len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    print(f'Epoch {epoch}/{EPOCHS} | Test Loss: {avg_test_loss:.4f} | Accuracy: {accuracy:.2f}%')


FashionCNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=10, bias=True)
  )
)
Epoch 1/10 | Test Loss: 0.0058 | Accuracy: 86.08%
Epoch 2/10 | Test Loss: 0.0050 | Accuracy: 88.28%
Epoch 3/10 | Test Loss: 0.0039 | Accuracy: 90.92%
Epoch 4/10 | Test Loss: 0.0046 | Accuracy: 89.41%
Epoch 5/10 | Test Loss: 0.0037 | Accuracy: 91.55%
Epoch 6/10 | Test Loss: 0.0037 | Accuracy: 92.22%
Epoch 7/10 | Test Loss: 0.0038 | Accuracy: 91.85%
Epoch 8/10 | Test Loss: 0.0040 | Accuracy: 

**Keras**

In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

train_images = (train_images - 0.5) / 0.5
test_images = (test_images - 0.5) / 0.5

train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)

input_shape = train_images.shape[1:]

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same', name='Conv1'),
    MaxPooling2D((2, 2), name='Pool1'),

    Conv2D(64, (3, 3), activation='relu', padding='same', name='Conv2'),
    MaxPooling2D((2, 2), name='Pool2'),

    Flatten(name='Flatten'),

    Dense(128, activation='relu', name='Dense1'),
    Dropout(0.5, name='Dropout'),
    Dense(10, activation='softmax', name='Output')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

history = model.fit(train_images, train_labels, epochs=10, batch_size=64, validation_split=0.1, verbose=1)

loss, accuracy = model.evaluate(test_images, test_labels, verbose=0)

print(f"Test Accuracy: {accuracy:.4f}")

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Conv1 (Conv2D)                  │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pool1 (MaxPooling2D)            │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2 (Conv2D)                  │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pool2 (MaxPooling2D)            │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense1 (Dense)                  │ (None, 128)            │       401,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 421,642 (1.61 MB)

 Trainable params: 421,642 (1.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 70s 80ms/step - accuracy: 0.7150 - loss: 0.8049 - val_accuracy: 0.8702 - val_loss: 0.3502
Epoch 2/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 66s 78ms/step - accuracy: 0.8643 - loss: 0.3858 - val_accuracy: 0.8890 - val_loss: 0.2987
Epoch 3/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 65s 77ms/step - accuracy: 0.8840 - loss: 0.3252 - val_accuracy: 0.9043 - val_loss: 0.2658
Epoch 4/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 66s 78ms/step - accuracy: 0.8994 - loss: 0.2853 - val_accuracy: 0.9143 - val_loss: 0.2384
Epoch 5/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 65s 77ms/step - accuracy: 0.9077 - loss: 0.2544 - val_accuracy: 0.9050 - val_loss: 0.2515
Epoch 6/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 85s 81ms/step - accuracy: 0.9132 - loss: 0.2389 - val_accuracy: 0.9117 - val_loss: 0.2294
Epoch 7/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 66s 78ms/step - accuracy: 0.9191 - loss: 0.2187 - val_accuracy: 0.9200 - val_loss: 0.2202
Epoch 8/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 66s 79ms/step - accuracy: 0.9268 - loss: 0.2003 - 